In [ ]:
!pip install datamol
!pip install rdkit-pypi
!pip install pandas
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=8218fdb1d394d1ea37ffda4b06ecb0a34280e3096baacd50db95a8ab703a4443
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from rdkit.Chem import rdmolops
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Linear
from scipy.stats import spearmanr

In [ ]:
from torch.nn import BatchNorm1d
from torch_geometric.nn import GCNConv, GlobalAttention
from torch_geometric.nn import global_add_pool, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from scipy.stats import spearmanr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Project/ALVS")

Mounted at /content/drive


In [ ]:
def one_hot(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def get_bond_pair(mol):
    bonds = mol.GetBonds()
    res = [[],[]]
    for bond in bonds:
        res[0] += [bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()]
        res[1] += [bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()]
    return res


def get_atom_features(mol):
    acceptor_smarts_one = '[!$([#1,#6,F,Cl,Br,I,o,s,nX3,#7v5,#15v5,#16v4,#16v6,*+1,*+2,*+3])]'
    acceptor_smarts_two = "[$([O,S;H1;v2;!$(*-*=[O,N,P,S])]),$([O,S;H0;v2]),$([O,S;-]),$([N;v3;!$(N-*=[O,N,P,S])]),n&H0&+0,$([o,s;+0;!$([o,s]:n);!$([o,s]:c:n)])]"
    donor_smarts_one = "[$([N;!H0;v3,v4&+1]),$([O,S;H1;+0]),n&H1&+0]"
    donor_smarts_two = "[!$([#6,H0,-,-2,-3]),$([!H0;#7,#8,#9])]"

    hydrogen_donor_one = Chem.MolFromSmarts(donor_smarts_one)
    hydrogen_donor_two = Chem.MolFromSmarts(donor_smarts_two)
    hydrogen_acceptor_one = Chem.MolFromSmarts(acceptor_smarts_one)
    hydrogen_acceptor_two = Chem.MolFromSmarts(acceptor_smarts_two)

    hydrogen_donor_match_one = mol.GetSubstructMatches(hydrogen_donor_one)
    hydrogen_donor_match_two = mol.GetSubstructMatches(hydrogen_donor_two)
    hydrogen_donor_match = []
    hydrogen_donor_match.extend(hydrogen_donor_match_one)
    hydrogen_donor_match.extend(hydrogen_donor_match_two)
    hydrogen_donor_match = list(set(hydrogen_donor_match))

    hydrogen_acceptor_match_one = mol.GetSubstructMatches(hydrogen_acceptor_one)
    hydrogen_acceptor_match_two = mol.GetSubstructMatches(hydrogen_acceptor_two)
    hydrogen_acceptor_match = []
    hydrogen_acceptor_match.extend(hydrogen_acceptor_match_one)
    hydrogen_acceptor_match.extend(hydrogen_acceptor_match_two)
    hydrogen_acceptor_match = list(set(hydrogen_acceptor_match))

    ring = mol.GetRingInfo()

    m = []
    for atom_idx in range(mol.GetNumAtoms()):
        atom = mol.GetAtomWithIdx(atom_idx)

        o = []
        o += one_hot(atom.GetSymbol(), ['C', 'H', 'O', 'N', 'S', 'Cl', 'F', 'Br', 'P',
                                        'I'])
        o += [atom.GetDegree()]
        o += one_hot(atom.GetHybridization(), [Chem.rdchem.HybridizationType.SP,
                                               Chem.rdchem.HybridizationType.SP2,
                                               Chem.rdchem.HybridizationType.SP3,
                                               Chem.rdchem.HybridizationType.SP3D,
                                               Chem.rdchem.HybridizationType.SP3D2])
        o += [atom.GetImplicitValence()]
        o += [atom.GetIsAromatic()]
        o += [ring.IsAtomInRingOfSize(atom_idx, 3),
              ring.IsAtomInRingOfSize(atom_idx, 4),
              ring.IsAtomInRingOfSize(atom_idx, 5),
              ring.IsAtomInRingOfSize(atom_idx, 6),
              ring.IsAtomInRingOfSize(atom_idx, 7),
              ring.IsAtomInRingOfSize(atom_idx, 8)]

        o += [atom_idx in hydrogen_donor_match]
        o += [atom_idx in hydrogen_acceptor_match]
        o += [atom.GetFormalCharge()]
        m.append(o)
    return m


def mol2vec(mol, score=None):
    node_f = get_atom_features(mol)
    edge_index = get_bond_pair(mol)

    data = Data(x=torch.tensor(node_f, dtype=torch.float32),
                edge_index=torch.tensor(edge_index, dtype=torch.long),
                score=torch.tensor([[score]], dtype=torch.float))
    return data

In [ ]:
def generate_datasets(df, test_size):
    datasets = []
    for idx, row in df.iterrows():
        mol = Chem.MolFromSmiles(row[1])
        score = row[2]
        if not mol:
            continue
        data = mol2vec(mol, score=score)
        datasets.append(data)

    train_dataset, valid_dataset = train_test_split(datasets, test_size=test_size)
    return train_dataset, valid_dataset

In [ ]:
def graph_from_smiles(smi):
    mol = Chem.MolFromSmiles(smi)
    if not mol:
        return np.nan
    node_f = get_atom_features(mol)
    edge_index = get_bond_pair(mol)

    batch = np.zeros(len(node_f), )
    data = Data(x=torch.tensor(node_f, dtype=torch.float32),
                    edge_index=torch.tensor(edge_index, dtype=torch.long),
                    batch=torch.tensor(batch, dtype=torch.long))
    return data

In [ ]:
n_features = 27
hidden = 1024

class GCNNet(torch.nn.Module):
    def __init__(self):
        super(GCNNet, self).__init__()
        self.conv1 = GCNConv(n_features, 1024, cached=False) # if you defined cache=True, the shape of batch must be same!
        self.bn1 = BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(p=0.2)
        self.conv2 = GCNConv(1024, 512, cached=False)
        self.bn2 = BatchNorm1d(512)
        self.dropout2 = nn.Dropout(p=0.2)
        self.conv3 = GCNConv(512, 256, cached=False)
        self.bn3 = BatchNorm1d(256)
        self.dropout3 = nn.Dropout(p=0.2)
        self.conv4 = GCNConv(256, 512, cached=False)
        self.bn4 = BatchNorm1d(512)
        self.dropout4 = nn.Dropout(p=0.2)
        self.conv5 = GCNConv(512, 1024, cached=False)
        self.bn5 = BatchNorm1d(1024)
        self.dropout5 = nn.Dropout(p=0.2)

        # self.att = GlobalAttention(Linear(hidden, 1))
        self.fc2 = Linear(1024, 128)
        self.dropout6 = nn.Dropout(p=0.2)
        self.fc3 = Linear(128, 16)
        self.dropout7 = nn.Dropout(p=0.2)
        self.fc4 = Linear(16, 1)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()

        self.att.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = self.dropout1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = self.dropout2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = self.dropout3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        x = self.dropout4(x)
        x = F.relu(self.conv5(x, edge_index))
        x = self.bn5(x)
        x = self.dropout5(x)
        x = global_mean_pool(x, batch)

        x = F.relu(self.fc2(x))
        x = self.dropout6(x)
        x = F.relu(self.fc3(x))
        x = self.dropout7(x)
        x = self.fc4(x)
        return x

In [ ]:
def train_epoch(loader, model, optimizer, device):
    model.train()

    loss_all = 0
    i = 0
    for data in loader:
        data = data.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, data.score)
        loss.backward()

        loss_all += loss.item()
        optimizer.step()
        i += 1
    return loss_all / i

In [ ]:
def test_epoch(loader, model,  device):
    model.eval()

    MSE, MAE = 0, 0
    trues, preds = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)

            output = model(data)
            pred = output.cpu().squeeze().numpy().tolist()
            true = data.score.cpu().squeeze().numpy().tolist()

            trues.extend(true)
            preds.extend(pred)
    MAE = mean_absolute_error(trues, preds)
    RMSE = np.sqrt(mean_squared_error(trues, preds))
    R2 = r2_score(trues, preds)
    Sp = spearmanr(trues, preds)[0]
    return MAE, RMSE, R2, Sp

In [ ]:
def init_model(device, lr=0.0001):
    model = GCNNet()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    return model, optimizer

In [ ]:
def prepare_dataloader(dff, batch_size, test_size=0.2):
    train_dataset, valid_dataset = generate_datasets(dff, test_size)
    train_loader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=batch_size,
                              shuffle=False)
    return train_loader, valid_loader

In [ ]:
def train_step(dff, epochs, batch_size, step_time, device="cuda"):
    model, optimizer = init_model(device=device)
    train_loader, valid_loader = prepare_dataloader(dff,  batch_size=batch_size)

    model_folder = "models/ada17/gnn/models_random/{}".format(step_time)
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)

    hist = {"train-loss":[], "test-mae":[], "test-rmse":[], "test-r2":[], "test-sp":[]}
    for epoch in range(epochs):
        train_loss = train_epoch(train_loader, model, optimizer, device)
        test_mae, test_rmse, test_r2, test_sp = test_epoch(valid_loader, model, device)
        hist["train-loss"].append(train_loss)
        hist["test-mae"].append(test_mae)
        hist["test-rmse"].append(test_rmse)
        hist["test-r2"].append(test_r2)
        hist["test-sp"].append(test_sp)

        if test_rmse <= min(hist["test-rmse"]):
            weight_path = os.path.join(model_folder, "weight_{}.pth".format(epoch))
            torch.save(model.state_dict(), weight_path)

        # print(f'Epoch: {epoch}, Train loss: {train_loss:.3}, Test mae: {test_mae:.3}, Test rmse: {test_rmse:.3}, Test r2: {test_r2:.3}')
    print("---------------------------------\nvalidation min rmse: {}\n---------------------------------\n".format(min(hist["test-rmse"])))
    return weight_path

In [ ]:
def random_select(df, n_samples):
    df_select = df.sample(n=n_samples).copy()
    sample_indexs = df_select["name"].tolist()
    df_remaining = df[~df["name"].isin(sample_indexs)].copy()
    return df_select, df_remaining

In [ ]:
def load_model(best_model_path, device="cuda"):
    model= GCNNet().to(device)
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.eval()
    return model

In [ ]:
def predict_with_uncertainty(data, model, device, times=10):
    data = data.to(device)
    dropout_predictions = []
    with torch.no_grad():
        for _ in range(times):
            output = model(data)
            pred = output.cpu().numpy()[0][0]
            dropout_predictions.append(pred)
    mean = np.mean(dropout_predictions)
    variance = np.var(dropout_predictions)
    return [mean, variance]

In [ ]:
def uncertainty_select(dff, best_model_path, n_samples):
    model = load_model(best_model_path)
    dff["mean_var"] = dff["graph"].map(lambda x: predict_with_uncertainty(x,  model, device="cuda"))
    dff["mean"] = dff["mean_var"].map(lambda x: x[0])
    dff["var"] = dff["mean_var"].map(lambda x: x[1])
    dff_sort = dff.sort_values(by="var", ascending=False)
    dff_select = dff_sort.iloc[:n_samples, :4].copy()
    sample_indexs = dff_select["name"].tolist()
    dff_remaining = dff_sort[~dff_sort["name"].isin(sample_indexs)].copy()
    return dff_select, dff_remaining

In [ ]:
# def predict(data, model, device):
#     data = data.to(device)
#     with torch.no_grad():
#             output = model(data)
#             pred = output.cpu().numpy()[0][0]
#     return pred

def predict(dff, model, batch_size=128, device="cuda"):
    preds = []
    loader = DataLoader(dff["graph"].tolist(), batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            pred = output.cpu().squeeze().numpy().tolist()
            preds.extend(pred)
    dff["pred"] = preds
    return dff

In [ ]:
def eval_model(df_test, best_model_path):
    model = load_model(best_model_path)
    # df_test["pred"] = df_test["graph"].map(lambda x: predict(x,  model, device="cuda"))
    df_test = predict(df_test, model, batch_size=1024, device="cuda")

    MAE = mean_absolute_error(df_test["score"], df_test["pred"])
    RMSE = np.sqrt(mean_squared_error(df_test["score"], df_test["pred"]))
    R2 = r2_score(df_test["score"], df_test["pred"])
    Sp = spearmanr(df_test["score"], df_test["pred"])[0]
    return MAE, RMSE, R2, Sp

In [ ]:
def run_step(dfs, epochs, stime):
    dfs_new = dfs.copy()

    best_model_path = train_step(dfs_new, epochs, batch_size=32, step_time=stime)
    return best_model_path

In [ ]:
def run_iterations(df, df_test, epochs=200, iterations=100, n_samples=100):
    results = {"RMSE": [], "R2": [], "Sp": []}

    print("len(df):", len(df))

    for stime in range(iterations):
        if stime < 1:
            dfs, dfr = random_select(df, n_samples)
        else:
            df_temp = dfs.copy()
            dfs, dfr = random_select(dfr, n_samples)
            dfs = pd.concat([dfs, df_temp], axis=0)
            dfs = dfs.reset_index(drop=True)
        print("number of mol pool:", len(dfs))
        best_model_path = run_step(dfs, epochs, stime)
        MAE, RMSE, R2 ,Sp = eval_model(df_test, best_model_path)
        print("MAE:{}, RMSE:{}, R2:{}, Sp:{},step: {}".format(MAE, RMSE, R2, Sp, stime))

        results["RMSE"].append(RMSE)
        results["R2"].append(R2)
        results["Sp"].append(Sp)

    # Convert results dictionary to DataFrame
    results_df = pd.DataFrame(results)

    results_df.to_csv('gnn_random_results.csv', index=False)

In [ ]:
df = pd.read_csv("preprocess_data/ada17/ada17_train.csv", sep="\t")
df["graph"] = df["smiles"].map(lambda x: graph_from_smiles(x))
df = df[~df["graph"].isna()]
df_test = pd.read_csv("preprocess_data/ada17/ada17_test.csv", sep="\t")
df_test["graph"] = df_test["smiles"].map(lambda x: graph_from_smiles(x))
df_test = df_test[~df_test["graph"].isna()]

In [ ]:

run_iterations(df, df_test)

len(df): 20479
number of mol pool: 100
---------------------------------
validation min rmse: 2.53120141941051
---------------------------------

MAE:1.9515297544510437, RMSE:2.817500833333951, R2:-7.49067347237628, Sp:0.10870762479465435,step: 0
number of mol pool: 200
---------------------------------
validation min rmse: 2.8904900838092695
---------------------------------

MAE:1.7654395375490057, RMSE:2.5453712481030895, R2:-5.9297298025056575, Sp:0.14090635906456242,step: 1
number of mol pool: 300
---------------------------------
validation min rmse: 1.5391631747155385
---------------------------------

MAE:1.3301437868688348, RMSE:2.0285710740759546, R2:-3.401437001001683, Sp:0.29355730780126105,step: 2
number of mol pool: 400
---------------------------------
validation min rmse: 1.3037816251669034
---------------------------------

MAE:1.1252134722991698, RMSE:1.6222478465128607, R2:-1.8148058551505857, Sp:0.34186954299479844,step: 3
number of mol pool: 500
-------------------

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-d7f2db78b0b6>", line 1, in <cell line: 1>
    run_iterations(df, df_test)
  File "<ipython-input-21-ef2971028607>", line 16, in run_iterations
    MAE, RMSE, R2 ,Sp = eval_model(df_test, best_model_path)
  File "<ipython-input-19-3f365cae7a5e>", line 2, in eval_model
    model = load_model(best_model_path)
  File "<ipython-input-15-82293f0fa416>", line 3, in load_model
    model.load_state_dict(torch.load(best_model_path, map_location=device))
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 271, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packa